# VacationPy


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data_to_load = "../weatherpy/output_data/CityWeatherData.csv"
weather_data = pd.read_csv(weather_data_to_load)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# configure gmaps
gmaps.configure(api_key=g_key)

# locations
locations = weather_data[["Lat", "Lng"]]

humidity = weather_data["Humidity"].astype(float)

In [10]:

# plot Heatmap
fig = gmaps.figure()

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
# filter to max temp below 80 degrees F
adjust1 = weather_data[weather_data["Max Temp"] < 80]
# filter to max temp above 70 degrees F
adjust2 = adjust1[adjust1["Max Temp"] > 70]
# filter to wind speed below 10 mph
adjust3 = adjust2[adjust2["Windspeed"] < 10]
# filter to zero cloudiness
adjust4 = adjust3[adjust3["Cloudiness"] == 0]

# reset index
ideal_weather = adjust4.reset_index()
del ideal_weather["index"]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,ottawa,42.95,-86.00,73.99,68,0,5.68,US,1592533195
1,dusti,37.35,68.66,77.00,25,0,4.47,TJ,1592533196
2,awbari,26.59,12.78,77.77,17,0,9.57,LY,1592533197
3,broome,-17.97,122.23,77.00,38,0,8.05,AU,1592533094
4,neon karlovasion,37.79,26.70,71.60,78,0,5.59,GR,1592533203
5,kysyl-syr,63.90,122.76,77.79,40,0,4.63,RU,1592533080
6,ravar,31.27,56.81,79.23,16,0,2.39,IR,1592533210
7,zhigansk,66.77,123.37,74.95,32,0,9.78,RU,1592533058
8,shieli,44.17,66.75,77.47,18,0,3.44,KZ,1592533211
9,noblesville,40.05,-86.01,75.99,65,0,5.86,US,1592532968


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df=ideal_weather.copy()

hotels = []

# find nearest hotel
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]

    city_coords = (f"{lat},{lng}")

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Nearest Hotel"] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Nearest Hotel
0,ottawa,42.95,-86.00,73.99,68,0,5.68,US,1592533195,River Pines RV Park Campground
1,dusti,37.35,68.66,77.00,25,0,4.47,TJ,1592533196,Hotel Izzat
2,awbari,26.59,12.78,77.77,17,0,9.57,LY,1592533197,فندق أوباري
3,broome,-17.97,122.23,77.00,38,0,8.05,AU,1592533094,Mangrove Hotel
4,neon karlovasion,37.79,26.70,71.60,78,0,5.59,GR,1592533203,Hotel Aegeon
5,kysyl-syr,63.90,122.76,77.79,40,0,4.63,RU,1592533080,Nearest hotel not found
6,ravar,31.27,56.81,79.23,16,0,2.39,IR,1592533210,computertik
7,zhigansk,66.77,123.37,74.95,32,0,9.78,RU,1592533058,Gostinichnyy Kompleks Aylgy
8,shieli,44.17,66.75,77.47,18,0,3.44,KZ,1592533211,Gostinitsa Aybek Shieli
9,noblesville,40.05,-86.01,75.99,65,0,5.86,US,1592532968,Quality Inn Noblesville-Indianapolis


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))